In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1️⃣ Build a Simple CNN Model
def create_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name="conv1"),
        layers.MaxPooling2D((2, 2), name="pool1"),
        layers.Conv2D(64, (3, 3), activation='relu', name="conv2"),
        layers.MaxPooling2D((2, 2), name="pool2"),
        layers.Flatten(name="flatten"),
        layers.Dense(128, activation='relu', name="dense1"),
        layers.Dense(10, activation='softmax', name="output")
    ])
    return model

# 2️⃣ Load Dataset (MNIST)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 3️⃣ Train the Baseline Model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🔹 Baseline Model Summary:")
model.summary()

print("\n🚀 Training the baseline model...")
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 4️⃣ Evaluate the Baseline Model
baseline_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Baseline Model Accuracy: {baseline_accuracy:.4f}")

# 5️⃣ Define Filter Pruning Function
def prune_conv_layer(conv_layer, prune_ratio=0.5):
    """Prunes filters with the lowest L1 norm from a Conv2D layer."""
    weights = conv_layer.get_weights()
    kernel, bias = weights[0], weights[1]

    num_filters = kernel.shape[-1]  # Number of filters

    # Compute L1 norm of each filter
    l1_norms = np.sum(np.abs(kernel), axis=(0, 1, 2))

    # Get indices of filters to KEEP
    num_prune = int(num_filters * prune_ratio)
    keep_indices = np.argsort(l1_norms)[num_prune:]

    # Keep only selected filters
    new_kernel = kernel[..., keep_indices]
    new_bias = bias[keep_indices]

    # Create a new Conv2D layer with fewer filters
    new_conv_layer = tf.keras.layers.Conv2D(
        filters=len(keep_indices),
        kernel_size=conv_layer.kernel_size,
        strides=conv_layer.strides,
        padding=conv_layer.padding,
        activation=conv_layer.activation,
        use_bias=(bias is not None),
        name=conv_layer.name + "_pruned"
    )

    # Apply new weights to the layer
    new_conv_layer.build(conv_layer.input.shape)
    new_conv_layer.set_weights([new_kernel, new_bias])

    return new_conv_layer, len(keep_indices)

# 6️⃣ Apply Pruning to the First Convolutional Layer
pruned_conv1, new_filters = prune_conv_layer(model.layers[0], prune_ratio=0.8)

# 7️⃣ Modify Pooling Layer and Next Convolutional Layer
pruned_pool1 = layers.MaxPooling2D((2, 2), name="pool1_pruned")

# **FIX: Explicitly define input shape for conv2_pruned**
pruned_conv2 = layers.Conv2D(
    filters=64, kernel_size=(3, 3), activation='relu',
    input_shape=(13, 13, new_filters),  # **Ensure correct input shape**
    name="conv2_pruned"
)

pruned_pool2 = layers.MaxPooling2D((2, 2), name="pool2_pruned")

# **FIX: Ensure Flatten layer adapts to new shape**
pruned_flatten = layers.Flatten(name="flatten_pruned")

# 8️⃣ Rebuild Model with Pruned Layers
new_model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),  # **Ensure input shape is set**
    pruned_conv1,
    pruned_pool1,
    pruned_conv2,
    pruned_pool2,
    pruned_flatten,
    model.layers[5],  # Dense
    model.layers[6],  # Output Layer
])

# 9️⃣ Show Pruned Model Summary
print("\n🔹 Pruned Model Summary:")
new_model.build(input_shape=(None, 28, 28, 1))  # **Ensure layers are built**
new_model.summary()

# 🔟 Compile and Retrain the Pruned Model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🚀 Retraining the pruned model...")
new_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 🔟 Evaluate the Pruned Model
pruned_accuracy = new_model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Pruned Model Accuracy: {pruned_accuracy:.4f}")

# 🔍 Compare Results
print(f"\n📊 Accuracy Before Pruning: {baseline_accuracy:.4f}")
print(f"📊 Accuracy After Pruning: {pruned_accuracy:.4f}")



🔹 Baseline Model Summary:


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                       │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1 (MaxPooling2D)                 │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2 (Conv2D)                       │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2 (MaxPooling2D)                 │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense1 (Dense)                       │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1_pruned (Conv2D)                │ (None, 26, 26, 7)           │              70 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1_pruned (MaxPooling2D)          │ (None, 13, 13, 7)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2_pruned (Conv2D)                │ (None, 11, 11, 64)          │           4,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2_pruned (MaxPooling2D)          │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_pruned (Flatten)             │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense1 (Dense)                       │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 210,384 (821.81 KB)

 Trainable params: 210,384 (821.81 KB)

 Non-trainable params: 0 (0.00 B)


🚀 Retraining the pruned model...
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 38s 19ms/step - accuracy: 0.9528 - loss: 0.1670 - val_accuracy: 0.9868 - val_loss: 0.0377
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 19ms/step - accuracy: 0.9891 - loss: 0.0336 - val_accuracy: 0.9898 - val_loss: 0.0302
✅ Pruned Model Accuracy: 0.9898

📊 Accuracy Before Pruning: 0.9887
📊 Accuracy After Pruning: 0.9898


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1️⃣ Build a Simple CNN Model
def create_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name="conv1"),
        layers.MaxPooling2D((2, 2), name="pool1"),
        layers.Conv2D(64, (3, 3), activation='relu', name="conv2"),
        layers.MaxPooling2D((2, 2), name="pool2"),
        layers.Flatten(name="flatten"),
        layers.Dense(128, activation='relu', name="dense1"),
        layers.Dense(10, activation='softmax', name="output")
    ])
    return model

# 2️⃣ Load Dataset (MNIST)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 3️⃣ Train the Baseline Model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🔹 Baseline Model Summary:")
model.summary()

print("\n🚀 Training the baseline model...")
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 4️⃣ Evaluate the Baseline Model
baseline_accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Baseline Model Accuracy: {baseline_accuracy:.4f}")

# 5️⃣ Define Filter Pruning Function
def prune_conv_layer(conv_layer, prune_ratio=0.5):
    """Prunes filters with the lowest L1 norm from a Conv2D layer."""
    weights = conv_layer.get_weights()
    kernel, bias = weights[0], weights[1]

    num_filters = kernel.shape[-1]  # Number of filters

    # Compute L1 norm of each filter
    l1_norms = np.sum(np.abs(kernel), axis=(0, 1, 2))

    # Get indices of filters to KEEP
    num_prune = int(num_filters * prune_ratio)
    keep_indices = np.argsort(l1_norms)[num_prune:]

    # Keep only selected filters
    new_kernel = kernel[..., keep_indices]
    new_bias = bias[keep_indices]

    # Create a new Conv2D layer with fewer filters
    new_conv_layer = layers.Conv2D(
        filters=len(keep_indices),
        kernel_size=conv_layer.kernel_size,
        strides=conv_layer.strides,
        padding=conv_layer.padding,
        activation=conv_layer.activation,
        use_bias=True,
        name=conv_layer.name + "_pruned"
    )

    # Apply new weights to the layer
    input_depth = kernel.shape[2]  # Preserve input depth
    new_conv_layer.build((None, None, None, input_depth))
    new_conv_layer.set_weights([new_kernel, new_bias])

    return new_conv_layer, len(keep_indices)

# 6️⃣ Apply Pruning to the First Convolutional Layer
pruned_conv1, filters_conv1 = prune_conv_layer(model.layers[0], prune_ratio=0.8)

# 7️⃣ Modify Pooling Layer to Match the New Output
pruned_pool1 = layers.MaxPooling2D((2, 2), name="pool1_pruned")

# 8️⃣ Prune the Second Convolutional Layer to Match `conv1_pruned` Output Depth
original_conv2 = model.layers[2]  # Get the second Conv2D layer
pruned_conv2, filters_conv2 = prune_conv_layer(original_conv2, prune_ratio=0.5)

# 🔍 Fix Input Depth Mismatch
pruned_conv2 = layers.Conv2D(
    filters=filters_conv2,
    kernel_size=original_conv2.kernel_size,
    strides=original_conv2.strides,
    padding=original_conv2.padding,
    activation=original_conv2.activation,
    use_bias=True,
    name=original_conv2.name + "_pruned"
)

# Ensure the input shape matches the output of `pruned_conv1`
pruned_conv2.build((None, None, None, pruned_conv1.filters))

# 9️⃣ Modify Pooling Layer to Match the New Output
pruned_pool2 = layers.MaxPooling2D((2, 2), name="pool2_pruned")

# 🔟 Compute the Flattened Output Shape Dynamically
dummy_input = np.random.randn(1, 28, 28, 1).astype(np.float32)  # Sample input batch
intermediate_model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    pruned_conv1,
    pruned_pool1,
    pruned_conv2,
    pruned_pool2,
    layers.Flatten()
])

# Get the output shape of the Flatten layer dynamically
dummy_output = intermediate_model.predict(dummy_input)
flattened_dim = dummy_output.shape[1]  # Correct dimension for Dense layer

# 🔟 Rebuild Model with Pruned Layers
new_model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    pruned_conv1,
    pruned_pool1,
    pruned_conv2,
    pruned_pool2,
    layers.Flatten(),
    layers.Dense(128, activation='relu', name="dense1_pruned"),  # Fixing input shape
    layers.Dense(10, activation='softmax', name="output")  # Output Layer
])

# 🔟 Show Pruned Model Summary
print("\n🔹 Pruned Model Summary:")
new_model.build(input_shape=(None, 28, 28, 1))
new_model.summary()

# 🔟 Compile and Retrain the Pruned Model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("\n🚀 Retraining the pruned model...")
new_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

# 🔟 Evaluate the Pruned Model
pruned_accuracy = new_model.evaluate(x_test, y_test, verbose=0)[1]
print(f"✅ Pruned Model Accuracy: {pruned_accuracy:.4f}")

# 🔍 Compare Results
print(f"\n📊 Accuracy Before Pruning: {baseline_accuracy:.4f}")
print(f"📊 Accuracy After Pruning: {pruned_accuracy:.4f}")



🔹 Baseline Model Summary:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                       │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1 (MaxPooling2D)                 │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2 (Conv2D)                       │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2 (MaxPooling2D)                 │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense1 (Dense)                       │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 225,034 (879.04 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1_pruned (Conv2D)                │ (None, 26, 26, 7)           │              70 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1_pruned (MaxPooling2D)          │ (None, 13, 13, 7)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2_pruned (Conv2D)                │ (None, 11, 11, 32)          │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2_pruned (MaxPooling2D)          │ (None, 5, 5, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense1_pruned (Dense)                │ (None, 128)                 │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 105,936 (413.81 KB)

 Trainable params: 105,936 (413.81 KB)

 Non-trainable params: 0 (0.00 B)


🚀 Retraining the pruned model...
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 33s 17ms/step - accuracy: 0.9104 - loss: 0.3113 - val_accuracy: 0.9824 - val_loss: 0.0527
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9835 - loss: 0.0534 - val_accuracy: 0.9842 - val_loss: 0.0464
✅ Pruned Model Accuracy: 0.9842

📊 Accuracy Before Pruning: 0.9884
📊 Accuracy After Pruning: 0.9842


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input
import numpy as np

# Define the CNN model properly with an explicit Input layer
def create_model():
    model = Sequential([
        Input(shape=(28, 28, 1)),  # Explicit input layer
        Conv2D(32, (3, 3), activation='relu', name='conv1'),
        Flatten(),
        Dense(10, activation='softmax', name='output')
    ])
    return model

# Build and compile the model
model = create_model()
model.build(input_shape=(None, 28, 28, 1))  # Ensure the model is built
model.summary()  # Display model architecture

# Function to prune filters based on activation magnitudes
def prune_filters_by_activation(model, layer_name, retain_ratio=0.5):
    """
    Prunes filters of a given convolutional layer based on their activation magnitude.

    Parameters:
    - model: Trained Keras model
    - layer_name: Name of the layer to be pruned
    - retain_ratio: Ratio of filters to retain (default: 50%)

    Returns:
    - New weights and biases after pruning
    """
    layer = model.get_layer(layer_name)
    weights, biases = layer.get_weights()

    # Compute mean activation magnitude of each filter
    filter_magnitudes = np.mean(np.abs(weights), axis=(0, 1, 2))  # Shape: (num_filters,)

    # Sort filters and keep only the top ones
    num_filters_to_keep = int(weights.shape[-1] * retain_ratio)
    top_filters = np.argsort(filter_magnitudes)[-num_filters_to_keep:]

    # Prune weights and biases
    pruned_weights = weights[:, :, :, top_filters]
    pruned_biases = biases[top_filters]

    return pruned_weights, pruned_biases, top_filters

# Prune 'conv1' layer and get pruned weights
pruned_weights, pruned_biases, selected_filters = prune_filters_by_activation(model, 'conv1')

# Rebuild the pruned model
def rebuild_pruned_model(selected_filters):
    new_model = Sequential([
        Input(shape=(28, 28, 1)),
        Conv2D(len(selected_filters), (3, 3), activation='relu', name='conv1'),  # Reduced filter count
        Flatten(),
        Dense(10, activation='softmax', name='output')
    ])
    return new_model

# Create new pruned model and assign new weights
pruned_model = rebuild_pruned_model(selected_filters)
pruned_model.build(input_shape=(None, 28, 28, 1))

# Set pruned weights to the new model
pruned_model.get_layer('conv1').set_weights([pruned_weights, pruned_biases])
pruned_model.summary()  # Check new model structure

print("Filter pruning complete. The new model has fewer filters in 'conv1'.")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                       │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 21632)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │         216,330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 216,650 (846.29 KB)

 Trainable params: 216,650 (846.29 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                       │ (None, 26, 26, 16)          │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 10816)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │         108,170 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 108,330 (423.16 KB)

 Trainable params: 108,330 (423.16 KB)

 Non-trainable params: 0 (0.00 B)

Filter pruning complete. The new model has fewer filters in 'conv1'.


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input
import numpy as np

# Load dataset (MNIST)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize data
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define the CNN model with an explicit Input layer
def create_model():
    model = Sequential([
        Input(shape=(28, 28, 1)),  # Explicit input layer
        Conv2D(32, (3, 3), activation='relu', name='conv1'),
        Flatten(),
        Dense(10, activation='softmax', name='output')
    ])
    return model

# Build and compile the model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the original model
print("Training original model...")
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Evaluate accuracy before pruning
_, accuracy_before = model.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy before pruning: {accuracy_before:.4f}")

# Function to prune filters based on activation magnitudes
def prune_filters_by_activation(model, layer_name, retain_ratio=0.5):
    """
    Prunes filters of a given convolutional layer based on their activation magnitude.

    Parameters:
    - model: Trained Keras model
    - layer_name: Name of the layer to be pruned
    - retain_ratio: Ratio of filters to retain (default: 50%)

    Returns:
    - New weights and biases after pruning
    """
    layer = model.get_layer(layer_name)
    weights, biases = layer.get_weights()

    # Compute mean activation magnitude of each filter
    filter_magnitudes = np.mean(np.abs(weights), axis=(0, 1, 2))  # Shape: (num_filters,)

    # Sort filters and keep only the top ones
    num_filters_to_keep = int(weights.shape[-1] * retain_ratio)
    top_filters = np.argsort(filter_magnitudes)[-num_filters_to_keep:]

    # Prune weights and biases
    pruned_weights = weights[:, :, :, top_filters]
    pruned_biases = biases[top_filters]

    return pruned_weights, pruned_biases, top_filters

# Prune 'conv1' layer and get pruned weights
pruned_weights, pruned_biases, selected_filters = prune_filters_by_activation(model, 'conv1')

# Rebuild the pruned model
def rebuild_pruned_model(selected_filters):
    new_model = Sequential([
        Input(shape=(28, 28, 1)),
        Conv2D(len(selected_filters), (3, 3), activation='relu', name='conv1'),  # Reduced filter count
        Flatten(),
        Dense(10, activation='softmax', name='output')
    ])
    return new_model

# Create new pruned model and assign new weights
pruned_model = rebuild_pruned_model(selected_filters)
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assign pruned weights to the new model
pruned_model.get_layer('conv1').set_weights([pruned_weights, pruned_biases])

# Fine-tune pruned model
print("Fine-tuning pruned model...")
pruned_model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))

# Evaluate accuracy after pruning
_, accuracy_after = pruned_model.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy after pruning: {accuracy_after:.4f}")

print("Filter pruning complete. The new model has fewer filters in 'conv1'.")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training original model...
Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 30s 30ms/step - accuracy: 0.8878 - loss: 0.3913 - val_accuracy: 0.9754 - val_loss: 0.0898
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.9779 - loss: 0.0741 - val_accuracy: 0.9785 - val_loss: 0.0703
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9856 - loss: 0.0497 - val_accuracy: 0.9813 - val_loss: 0.0603
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.9893 - loss: 0.0365 - val_accuracy: 0.9807 - val_loss: 0.0649
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9912 - loss: 0.0294 - val_accuracy: 0.9817 - val_loss: 0.0643
Accuracy before pruning: 0.9817
Fine-tuning pruned model...
Epoch 1/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - accuracy: 0.9198 - loss: 0.3035 - val_accuracy: 0.9750 - val_loss: 0.0826
Epoch 2/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.9806 - loss: 0.0687 -

Pruned and non pruned model save

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input
import numpy as np

# Load dataset (MNIST)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize data
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define the CNN model with an explicit Input layer
def create_model():
    model = Sequential([
        Input(shape=(28, 28, 1)),  # Explicit input layer
        Conv2D(32, (3, 3), activation='relu', name='conv1'),
        Flatten(),
        Dense(10, activation='softmax', name='output')
    ])
    return model

# Build and compile the model
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the original model
print("Training original model...")
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Save the original model before pruning
model.save("original_model.h5")
print("Original model saved as 'original_model.h5'.")

# Evaluate accuracy before pruning
_, accuracy_before = model.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy before pruning: {accuracy_before:.4f}")

# Function to prune filters based on activation magnitudes
def prune_filters_by_activation(model, layer_name, retain_ratio=0.5):
    """
    Prunes filters of a given convolutional layer based on their activation magnitude.

    Parameters:
    - model: Trained Keras model
    - layer_name: Name of the layer to be pruned
    - retain_ratio: Ratio of filters to retain (default: 50%)

    Returns:
    - New weights and biases after pruning
    """
    layer = model.get_layer(layer_name)
    weights, biases = layer.get_weights()

    # Compute mean activation magnitude of each filter
    filter_magnitudes = np.mean(np.abs(weights), axis=(0, 1, 2))  # Shape: (num_filters,)

    # Sort filters and keep only the top ones
    num_filters_to_keep = int(weights.shape[-1] * retain_ratio)
    top_filters = np.argsort(filter_magnitudes)[-num_filters_to_keep:]

    # Prune weights and biases
    pruned_weights = weights[:, :, :, top_filters]
    pruned_biases = biases[top_filters]

    return pruned_weights, pruned_biases, top_filters

# Prune 'conv1' layer and get pruned weights
pruned_weights, pruned_biases, selected_filters = prune_filters_by_activation(model, 'conv1')

# Rebuild the pruned model
def rebuild_pruned_model(selected_filters):
    new_model = Sequential([
        Input(shape=(28, 28, 1)),
        Conv2D(len(selected_filters), (3, 3), activation='relu', name='conv1'),  # Reduced filter count
        Flatten(),
        Dense(10, activation='softmax', name='output')
    ])
    return new_model

# Create new pruned model and assign new weights
pruned_model = rebuild_pruned_model(selected_filters)
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assign pruned weights to the new model
pruned_model.get_layer('conv1').set_weights([pruned_weights, pruned_biases])

# Fine-tune pruned model
print("Fine-tuning pruned model...")
pruned_model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))

# Evaluate accuracy after pruning
_, accuracy_after = pruned_model.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy after pruning: {accuracy_after:.4f}")

# Save the pruned model
pruned_model.save("pruned_model.h5")
print("Pruned model saved as 'pruned_model.h5'.")

print("Filter pruning complete. The new model has fewer filters in 'conv1'.")


Training original model...
Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - accuracy: 0.8926 - loss: 0.3816 - val_accuracy: 0.9722 - val_loss: 0.0978
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 43s 24ms/step - accuracy: 0.9783 - loss: 0.0783 - val_accuracy: 0.9797 - val_loss: 0.0677
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.9837 - loss: 0.0547 - val_accuracy: 0.9795 - val_loss: 0.0622
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.9883 - loss: 0.0414 - val_accuracy: 0.9801 - val_loss: 0.0638
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 23ms/step - accuracy: 0.9901 - loss: 0.0327 - val_accuracy: 0.9802 - val_loss: 0.0676


Original model saved as 'original_model.h5'.
Accuracy before pruning: 0.9802
Fine-tuning pruned model...
Epoch 1/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - accuracy: 0.9200 - loss: 0.3009 - val_accuracy: 0.9752 - val_loss: 0.0751
Epoch 2/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9805 - loss: 0.0656 - val_accuracy: 0.9796 - val_loss: 0.0668
Epoch 3/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step - accuracy: 0.9867 - loss: 0.0466 - val_accuracy: 0.9805 - val_loss: 0.0627


Accuracy after pruning: 0.9805
Pruned model saved as 'pruned_model.h5'.
Filter pruning complete. The new model has fewer filters in 'conv1'.
